In [17]:
import pandas as pd
import sys

# CSV 불러오기
df_qa = pd.read_csv("/home/food/people/minju/data/GPT_QA_40.csv")
file_path = "/home/food/people/subin/data/식품안전정보DB-url 추가(2014~2024).xls"
df_doc = pd.read_excel(file_path, sheet_name="2023")
# 문서 관련 빈 열 추가 (처음 실행 시만)
if 'document' not in df_qa.columns:
    df_qa["document"] = ""
    df_qa["title"] = ""
    df_qa["text"] = ""

def search_document_by_keyword(keyword, top_n=5):
    matches = df_doc[df_doc['제목'].str.contains(keyword, case=False, na=False)]
    return matches.head(top_n)

def assign_document_to_qa(qa_index, doc_row):
    df_qa.at[qa_index, 'title'] = doc_row['제목']
    df_qa.at[qa_index, 'text'] = doc_row['내용']
    df_qa.at[qa_index, 'document'] = f"{doc_row['제목']}\n\n{doc_row['내용']}"

# QA 전체 순회하며 빈 document 채우기
for i in range(len(df_qa)):
    if pd.notna(df_qa.at[i, "document"]) and df_qa.at[i, "document"] != "":
        continue  # 이미 채워진 경우 skip

    print(f"\n🔹 [{i+1}/40] QA 항목")
    print("질문:", df_qa.at[i, "query"])
    print("응답:", df_qa.at[i, "gpt4o_answer"])

    while True:
        keyword = input("\n[입력] 문서 검색 키워드 입력 (또는 'skip', 'exit') : ").strip()
        if keyword.lower() == 'skip':
            break
        elif keyword.lower() == 'exit':
            print("❗ 중단됨. 지금까지 저장한 내용 저장합니다.")
            df_qa.to_csv("final_gpt4o_qa_40_with_document_filled.csv", index=False)
            exit()

        results = search_document_by_keyword(keyword, top_n=7
                                             
                                             )

        if results.empty:
            print("⚠️  검색 결과가 없습니다. 다른 키워드를 입력하세요.")
            continue

        # 결과 미리보기
        for idx, row in results.iterrows():
            print(f"\n👉 문서 인덱스: {idx}")
            print("제목:", row['제목'])
            print("내용:", row['내용'][:300], "...")
            print("내용:", row['내용'][:300], "...", flush=True)
            sys.stdout.flush()  # 바로 다음 input 전에 flush

        sel = input("\n[선택] 사용할 문서 인덱스 입력 (또는 '다시', 'skip') : ").strip()
        if sel.lower() == 'skip':
            break
        elif sel.lower() == '다시':
            continue
        elif sel.isdigit() and int(sel) in results.index:
            assign_document_to_qa(i, df_doc.loc[int(sel)])
            print("✅ 문서가 해당 QA에 추가되었습니다.")
            break
        else:
            print("⚠️  잘못된 입력입니다. 다시 시도하세요.")

# 모두 끝나면 저장
df_qa.to_csv("final_gpt4o_qa_40_with_document_filled.csv", index=False)
print("\n✅ 전체 문서 저장 완료: final_gpt4o_qa_40_with_document_filled.csv") 


🔹 [1/40] QA 항목
질문: Waitrose 8 Red Onion Bhajis with a Date and Tamarind Dip 제품이 왜 회수되었나?
응답: 소비기한 오표시 때문
❗ 중단됨. 지금까지 저장한 내용 저장합니다.
⚠️  검색 결과가 없습니다. 다른 키워드를 입력하세요.
❗ 중단됨. 지금까지 저장한 내용 저장합니다.
⚠️  검색 결과가 없습니다. 다른 키워드를 입력하세요.
❗ 중단됨. 지금까지 저장한 내용 저장합니다.
⚠️  검색 결과가 없습니다. 다른 키워드를 입력하세요.
❗ 중단됨. 지금까지 저장한 내용 저장합니다.
⚠️  검색 결과가 없습니다. 다른 키워드를 입력하세요.

👉 문서 인덱스: 0
제목: [회수]영국 Chiltern Artisan, 대장균 오염으로 고추 스틱 회수
내용: - 일자 : 2023.12.30- 지역: 잉글랜드- 제품명 : Chiltern Artisan Chilli Sticks (모든 팩 사이즈, 소비기한(Use by)이 표시된 경우 모든 소비기한)- 회수사 : Chiltern Artisan- 회수사유 : 일부 제품의 대장균(E.coli) 오염 발견- 제품사진 : https://s3-eu-west-1.amazonaws.com/fsa-alerts-files/production/FSA-PRIN-52-2023/Customer-notice-Chiltern-Artisan-recalls-Chilli- ...
내용: - 일자 : 2023.12.30- 지역: 잉글랜드- 제품명 : Chiltern Artisan Chilli Sticks (모든 팩 사이즈, 소비기한(Use by)이 표시된 경우 모든 소비기한)- 회수사 : Chiltern Artisan- 회수사유 : 일부 제품의 대장균(E.coli) 오염 발견- 제품사진 : https://s3-eu-west-1.amazonaws.com/fsa-alerts-files/production/FSA-PRIN-52-2023/Customer-notice-Chiltern-Artisan-reca

: 

In [27]:
import pandas as pd

# 1. 문서 파일 불러오기
file_path = "/home/food/people/subin/data/식품안전정보DB-url 추가(2014~2024).xls"
df_doc = pd.read_excel(file_path, sheet_name="2023")

# # 2. QA CSV 불러오기
# df_qa = pd.read_csv("final_gpt4o_qa_40_with_document_filled.csv")  # 경로는 필요 시 수정

# 3. 문서 검색 함수
def search_doc_by_keyword(keyword):
    results = df_doc[df_doc['내용'].str.contains(keyword, case=False, na=False)]
    if results.empty:
        print("결과 없음")
    else:
        for idx, row in results.iterrows():
            print(f"\n[문서 인덱스: {idx}]")
            print("제목:", row["제목"])
            print("내용:", row["내용"][:300], "...")
    return results

# 4. 특정 QA 인덱스에 선택한 문서 삽입 함수
def assign_doc_to_qa(qa_idx, doc_idx):
    df_qa.at[qa_idx, 'title'] = df_doc.loc[doc_idx, '제목']
    df_qa.at[qa_idx, 'text'] = df_doc.loc[doc_idx, '내용']
    df_qa.at[qa_idx, 'document'] = f"{df_doc.loc[doc_idx, '제목']}\n\n{df_doc.loc[doc_idx, '내용']}"
    print("✅ 문서가 QA에 추가되었습니다.")

# ---------------------------
# 💡 사용 예시
# ---------------------------

# 예: 5번째 QA (인덱스 4)에 대해 "Arauco" 키워드로 검색 후 선택
# results = search_doc_by_keyword("나이지리아산 히비스커스 꽃")

# 결과 보고 괜찮은 문서 인덱스가 보이면 아래처럼 지정
assign_doc_to_qa(qa_idx=4, doc_idx=308)  # doc_idx는 실제 검색 결과 인덱스로 교체

# 저장
# df_qa.to_csv("final_gpt4o_qa_40_with_document_filled.csv", index=False)
# print("저장 완료")


✅ 문서가 QA에 추가되었습니다.


In [33]:
df_qa.to_csv("real_final.csv", index=False)


In [36]:
import pandas as pd
df_qa = pd.read_csv("/home/food/people/minju/data/real_final.csv")
df_qa['text'][1]


'[회수]일본 Hassui Kamaboko Co.,Ltd., 일부 상품 팽창 및 가스 고임으로 인한 부패로 어육연제품 회수-일시: 2023년 12월 26일-제품명: 구자쿠 아카(孔雀 赤) 1개/진공포장 > 상세: 계란을 어육으로 감싸 튀긴 연제품-JAN코드: 4976730003474-상미기한: 2024년 1월 20일-회수사유: 식품위생법 위반 우려 - 일부 상품에서 팽창하여 가스가 고여 부패 부분이 확인되었음 -판매지역: 오이타현 내-판매처: 오이타현 내 슈퍼-판매일: 2023년 12월 24일부터 12월 26일-판매수량: 1,388개-회수사: Hassui Kamaboko Co.,Ltd.(八水蒲鉾株式会社)-제조소 및 가공소: Hassui Kamaboko Co.,Ltd.(八水蒲鉾株式会社)*후생노동성 정리번호: RCL202303720'

In [ ]:
import pandas as pd
from transformers import AutoTokenizer

# 1. tokenizer 로드 (예: LLaMA 기준)
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3.1-8B-Instruct")


# 3. Excel 문서 로딩 (모든 시트)
excel_path = "/home/food/people/subin/data/식품안전정보DB-url 추가(2014~2024).xls"
sheet_names = pd.ExcelFile(excel_path).sheet_names

# 4. 본문 텍스트를 모두 수집
all_texts = []

# Excel 시트별 본문 컬럼 수집
for sheet in sheet_names:
    df_sheet = pd.read_excel(excel_path, sheet_name=sheet)
    
    # 본문 유력 컬럼명 중 하나 찾기
    for col in ["document", "내용", "본문", "extracted_text", "content"]:
        if col in df_sheet.columns:
            all_texts.extend(df_sheet[col].dropna().astype(str).tolist())
            break

# 5. 토큰 수 계산
total_tokens = sum(len(tokenizer.encode(text, add_special_tokens=False)) for text in all_texts)
total_docs = len(all_texts)

# 6. 평균 출력
if total_docs > 0:
    avg_token = total_tokens / total_docs
    print(f"총 문서 수: {total_docs}")
    print(f"총 토큰 수: {total_tokens}")
    print(f"📌 평균 토큰 수: {avg_token:.2f}")
else:
    print("본문을 가진 문서를 찾을 수 없습니다.")


In [1]:
import pandas as pd
import tiktoken

# GPT-4o 기준 tokenizer
tokenizer = tiktoken.get_encoding("cl100k_base")

# 파일 경로 및 연도 범위 설정
file_path = "/home/food/people/subin/data/식품안전정보DB-url 추가(2014~2024).xls"
start_year = 14
end_year = 24

# 전체 데이터프레임 초기화
full_df = pd.DataFrame()

# 각 시트에서 데이터 읽고 토큰 계산
for year in range(start_year, end_year + 1):
    sheet_name = f"20{year}"
    try:
        df = pd.read_excel(file_path, sheet_name=sheet_name, usecols=["제목", "내용"])
        df = df[~df["내용"].isnull()].reset_index(drop=True)
        df["제목_내용"] = df["제목"].astype(str) + " " + df["내용"].astype(str)
        df["year"] = sheet_name
        full_df = pd.concat([full_df, df], ignore_index=True)
    except Exception as e:
        print(f"[오류] {sheet_name} 시트: {e}")

# 문서별 토큰 수 계산
def count_tokens(text):
    return len(tokenizer.encode(text))

full_df["token_count"] = full_df["제목_내용"].apply(count_tokens)

# 연도별 평균 토큰 수 계산
avg_tokens_by_year = full_df.groupby("year")["token_count"].mean().reset_index()
avg_tokens_by_year.columns = ["year", "avg_token_count"]

# 결과 출력
print("\n[문서별 평균 토큰 수 (연도별)]")
print(avg_tokens_by_year)

# 전체 병합 결과 저장 (선택)
# full_df.to_csv("document_tokens_with_year.csv", index=False)
# avg_tokens_by_year.to_csv("average_token_count_by_year.csv", index=False)



[문서별 평균 토큰 수 (연도별)]
    year  avg_token_count
0   2014       567.586822
1   2015       557.186334
2   2016       489.348306
3   2017       498.676340
4   2018       494.193298
5   2019       494.522999
6   2020       494.310807
7   2021       415.437000
8   2022       438.396483
9   2023       402.475812
10  2024       447.981899


In [13]:
full_df

,제목,내용,제목_내용,year,token_count
0,"농림수산성, 고병원성 조류인플루엔자 NA아형 확정","12월 31일, 농림수산성은 야마구치현에서 발생한 고병원성 조류인플루엔자 의심사례에...","농림수산성, 고병원성 조류인플루엔자 NA아형 확정 12월 31일, 농림수산성은 야마...",2014,174
1,"장시성 가오안시 병사한 돼지고기 시장 유입 사건 관련, 전방위 조사 통해 사건 관계...",최근 장시(江西)성 가오안(高安)시의 병사한 돼지고기가 시장에 유입됐다는 사건이 언...,"장시성 가오안시 병사한 돼지고기 시장 유입 사건 관련, 전방위 조사 통해 사건 관계...",2014,597
2,"가오안시 병사한 돼지 사건 관련, 위산현으로 도주한 혐의자 체포",가오안(高安)시의 병사한 돼지 사건이 언론에서 보도된 후 관련 당국은 사건 혐의자들...,"가오안시 병사한 돼지 사건 관련, 위산현으로 도주한 혐의자 체포 가오안(高安)시의 ...",2014,251
3,"미국 공익과학센터, 농무부의 신선 식육 및 가금류 제품의 첨가용액 설명 규정에 관한...",미국 공익과학센터의 마이클 제이콥슨 대표는 신선 식육 및 가금류 제품의 첨가 용액 ...,"미국 공익과학센터, 농무부의 신선 식육 및 가금류 제품의 첨가용액 설명 규정에 관한...",2014,777
4,"원창시, 밀수 냉동육 1,227kg 압수",냉동육 시장 질서를 바로잡고 밀수 냉동육 위반 행위를 철저히 단속하기 위해 원창(文...,"원창시, 밀수 냉동육 1,227kg 압수 냉동육 시장 질서를 바로잡고 밀수 냉동육 ...",2014,550
...,...,...,...,...,...
215712,"홍콩 식품안전센터, 11월 식품안전보고 발표","식품환경위생서 식품안전센터는 금일(12월 29일), 11월 식품안전 보고를 발표함....","홍콩 식품안전센터, 11월 식품안전보고 발표 식품환경위생서 식품안전센터는 금일(12...",2024,845
215713,"홍콩 식품안전센터, 일본산 식품 검사 일일 개황('23년 12월 29일 정오 12시...","일본 정부가 2023년 8월 24일부터 원전 오염수 해양 방류를 개시함에 따라, 식...","홍콩 식품안전센터, 일본산 식품 검사 일일 개황('23년 12월 29일 정오 12시...",2024,528
215714,"마카오, 일본산 식품 방사능 검사 최신 데이터('23년 12월 30일 현재)",[일본산 식품 방사능 수준 검사 최신 데이터] 식품유형;이미 검사한 식품 샘플 수(...,"마카오, 일본산 식품 방사능 검사 최신 데이터('23년 12월 30일 현재) [일본...",2024,351
215715,"[회수]미국 Valley Meats, LLC, 대장균 O157:H7 오염 가능성으로...","-일자: 2023.12.31.-유통지역: 미국 Illinois, Indiana, I...","[회수]미국 Valley Meats, LLC, 대장균 O157:H7 오염 가능성으로...",2024,835


In [12]:
import pandas as pd
import tiktoken

# tokenizer 불러오기 (GPT-4o는 cl100k_base 사용)
tokenizer = tiktoken.get_encoding("cl100k_base")

# CSV 파일 경로
csv_path = "/home/food/people/minju/data/make_qa/real_final.csv"  # ← 여기에 실제 경로 입력
df = pd.read_csv(csv_path)

# inpu

# output: gpt4o_answer 기준
df["output_tokens"] = df["gpt4o_answer"].apply(lambda x: len(tokenizer.encode(str(x))) if pd.notnull(x) else 0)


df['output_tokens'].mean()


84.2

In [9]:
import tiktoken

def count_tokens(text, model_name="gpt-4"):
    tokenizer = tiktoken.get_encoding("cl100k_base")
    # encoding = tiktoken.encoding_for_model(model_name)
    return len(tokenizer.encode(text))

# Prompt 구성
def make_prompt():
    return f"""너는 문서 기반 QA 데이터를 생성하는 도우미야. 주어진 문서에서 질문(Query)과 질문에 대한 답변(Answer) 쌍을 추출해야 해. 특히 **가장 핵심적이거나 중요한 질문 1개만** 생성해야 하며, 이는 **사람이 이 문서를 봤을 때 가장 먼저 궁금해할 질문**이어야 해. 모든 출력은 반드시 한국어로 작성하시오.


### 출력 형식
[
 {{"query": "실제 문서 기반 질문", "answer": "문서에서 확인된 답변"}}
]

### 생성 규칙
1. 질문은 반드시 주어진 문서 내용을 바탕으로 사람이 실제로 궁금해할 만한 자연스러운 문장으로 작성하세요. 인위적인 패턴을 반복하지 마세요.
2. 답변은 반드시 해당 문서에서 직접 확인 가능한 정보만 포함해야 합니다. 문서에 없는 내용을 추측하거나 외삽하지 마세요.
3. 문서에서 답이 명확히 주어지지 않은 경우, 답변은 "해당 문서에 없음"으로 작성하세요.
4. 한 문서당 하나의 질문-답변 쌍만 생성하세요.
5. 질문은 서로 다른 정보에 초점을 맞춰 다양하게 구성하세요. 
   - 예: 제품명, 제조사, 유통 경로, 일자, 회수 사유, 법 위반 항목, 대표자 등
6. 하나의 질문에 너무 많은 요소를 담지 마세요. 질문은 간결하고 명확하게 하나의 정보만 물어보도록 하세요.
7. 질문은 완전한 문장으로 작성하세요. 단문이나 명령문 형태는 피하세요.
8. 질문 문장은 유사하거나 반복되지 않도록 하며, 사람스러운 문장 흐름을 갖추세요.
"""


# 최종 프롬프트 생성 및 토큰 계산
prompt_text=make_prompt()
token_count = count_tokens(prompt_text)

token_count


585

In [ ]:
import pandas as pd
import sys

# CSV 불러오기
df_qa = pd.read_csv("/home/food/people/minju/data/GPT_QA_40.csv")
file_path = "/home/food/people/subin/data/식품안전정보DB-url 추가(2014~2024).xls"
df_doc = pd.read_excel(file_path, sheet_name="2023")
# 문서 관련 빈 열 추가 (처음 실행 시만)
if 'document' not in df_qa.columns:
    df_qa["document"] = ""
    df_qa["title"] = ""
    df_qa["text"] = ""

def search_document_by_keyword(keyword, top_n=5):
    matches = df_doc[df_doc['제목'].str.contains(keyword, case=False, na=False)]
    return matches.head(top_n)

def assign_document_to_qa(qa_index, doc_row):
    df_qa.at[qa_index, 'title'] = doc_row['제목']
    df_qa.at[qa_index, 'text'] = doc_row['내용']
    df_qa.at[qa_index, 'document'] = f"{doc_row['제목']}\n\n{doc_row['내용']}"

# QA 전체 순회하며 빈 document 채우기
for i in range(len(df_qa)):
    if pd.notna(df_qa.at[i, "document"]) and df_qa.at[i, "document"] != "":
        continue  # 이미 채워진 경우 skip

    print(f"\n🔹 [{i+1}/40] QA 항목")
    print("질문:", df_qa.at[i, "query"])
    print("응답:", df_qa.at[i, "gpt4o_answer"])

    while True:
        keyword = input("\n[입력] 문서 검색 키워드 입력 (또는 'skip', 'exit') : ").strip()
        if keyword.lower() == 'skip':
            break
        elif keyword.lower() == 'exit':
            print("❗ 중단됨. 지금까지 저장한 내용 저장합니다.")
            df_qa.to_csv("final_gpt4o_qa_40_with_document_filled.csv", index=False)
            exit()

        results = search_document_by_keyword(keyword, top_n=7
                                             
                                             )

        if results.empty:
            print("⚠️  검색 결과가 없습니다. 다른 키워드를 입력하세요.")
            continue

        # 결과 미리보기
        for idx, row in results.iterrows():
            print(f"\n👉 문서 인덱스: {idx}")
            print("제목:", row['제목'])
            print("내용:", row['내용'][:300], "...")
            print("내용:", row['내용'][:300], "...", flush=True)
            sys.stdout.flush()  # 바로 다음 input 전에 flush

        sel = input("\n[선택] 사용할 문서 인덱스 입력 (또는 '다시', 'skip') : ").strip()
        if sel.lower() == 'skip':
            break
        elif sel.lower() == '다시':
            continue
        elif sel.isdigit() and int(sel) in results.index:
            assign_document_to_qa(i, df_doc.loc[int(sel)])
            print("✅ 문서가 해당 QA에 추가되었습니다.")
            break
        else:
            print("⚠️  잘못된 입력입니다. 다시 시도하세요.")

# 모두 끝나면 저장
df_qa.to_csv("final_gpt4o_qa_40_with_document_filled.csv", index=False)
print("\n✅ 전체 문서 저장 완료: final_gpt4o_qa_40_with_document_filled.csv") 


🔹 [1/40] QA 항목
질문: Waitrose 8 Red Onion Bhajis with a Date and Tamarind Dip 제품이 왜 회수되었나?
응답: 소비기한 오표시 때문
❗ 중단됨. 지금까지 저장한 내용 저장합니다.
⚠️  검색 결과가 없습니다. 다른 키워드를 입력하세요.
❗ 중단됨. 지금까지 저장한 내용 저장합니다.
⚠️  검색 결과가 없습니다. 다른 키워드를 입력하세요.
❗ 중단됨. 지금까지 저장한 내용 저장합니다.
⚠️  검색 결과가 없습니다. 다른 키워드를 입력하세요.
❗ 중단됨. 지금까지 저장한 내용 저장합니다.
⚠️  검색 결과가 없습니다. 다른 키워드를 입력하세요.

👉 문서 인덱스: 0
제목: [회수]영국 Chiltern Artisan, 대장균 오염으로 고추 스틱 회수
내용: - 일자 : 2023.12.30- 지역: 잉글랜드- 제품명 : Chiltern Artisan Chilli Sticks (모든 팩 사이즈, 소비기한(Use by)이 표시된 경우 모든 소비기한)- 회수사 : Chiltern Artisan- 회수사유 : 일부 제품의 대장균(E.coli) 오염 발견- 제품사진 : https://s3-eu-west-1.amazonaws.com/fsa-alerts-files/production/FSA-PRIN-52-2023/Customer-notice-Chiltern-Artisan-recalls-Chilli- ...
내용: - 일자 : 2023.12.30- 지역: 잉글랜드- 제품명 : Chiltern Artisan Chilli Sticks (모든 팩 사이즈, 소비기한(Use by)이 표시된 경우 모든 소비기한)- 회수사 : Chiltern Artisan- 회수사유 : 일부 제품의 대장균(E.coli) 오염 발견- 제품사진 : https://s3-eu-west-1.amazonaws.com/fsa-alerts-files/production/FSA-PRIN-52-2023/Customer-notice-Chiltern-Artisan-reca

: 